In [1]:
! pip install psycopg2

In [2]:
! python3 -m pip install requests

In [3]:
# import postgres
import psycopg2
import requests

In [4]:
# initiate connection

conn = psycopg2.connect(
    host = 'localhost'
    , user = 'ayo' # your username
    , password = 'password' # your password
    , database = 'bissan' # your database name
)

In [5]:
# create a cursor
cursor = conn.cursor()

## Question 1:
How many employees spent above their given stipend? Extract the name of the employee, the assigned stipend, and the actual amount spent.

In [10]:
import pandas as pd

In [25]:
# Extract employees and stipends data from db and compare

query = """
SELECT DISTINCT name, d.entitled_bonus as stipend 
FROM employees e
LEFT JOIN departments d
ON e.department_id = d._id
"""

cursor.execute(query)

In [26]:
all_employees_stipend = cursor.fetchall()

In [27]:
cursor.description #This will list the columns we are using i.e header of the extracted data from db

(Column(name='name', type_code=1043), Column(name='stipend', type_code=23))

In [32]:
employees_df = pd.DataFrame(all_employees_stipend, columns=['name', 'stipend'])

In [33]:
employees_df

,name,stipend
0,Robert Green,6832
1,James Sawyer,6832
2,Dieter Klinger,7377
3,Chase Fabrizio,6913
4,Scott Davis,6913
...,...,...
408,Lisa Killen,6086
409,Jason Giza,6832
410,Dana Stroman,7377
411,Joseph Webster,6832


In [35]:
employees_df[employees_df['name'].duplicated()]

,name,stipend
14,John Bentley,6086
133,Thomas Jordan,6086
146,Sean Renauer,6832
155,Mitsuko Herrera,6913
164,Catherine Brewer,6086
175,Sean Renauer,6913
320,Russ Hamill,6913
333,Dimitry Ruvin,6913
343,Darren Crandell,6832
375,Mark Sheelor,7377


In [36]:
#To get the actual amount spent, the info is to be extracted from the travel dataset saved in dir as cleaned_bissan_travels_data.csv

travels_dataset = pd.read_csv('cleaned_bissan_travels_data.csv') # read the csv

In [37]:
travels_dataset #The below shows that pandas indexed the data so we have double indexing, remove the index tagged as Unnamed: 0

,Unnamed: 0,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
0,0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City
1,1,Abigail Ratnofsky,2016-01-24,2016-01-29,2962.70,Conference,CA,San Diego
2,2,Adam Kisthardt,2018-10-04,2018-10-09,1891.19,Conference,FL,Orlando
3,3,Adam P. Jones,2016-06-19,2016-06-21,324.36,Conference,MD,Ocean City
4,4,Alan Butsch,2019-11-19,2019-11-22,2141.51,Conference,CA,San Diego
...,...,...,...,...,...,...,...,...
413,413,Warren Jensen,2017-11-06,2017-11-09,502.45,Conference,MD,Ocean City
414,414,Warp Jensen,2019-03-25,2019-03-26,78.00,Conference,MD,Ocean City
415,415,Whitney Kujawa,2017-11-06,2017-11-09,567.00,Conference,MD,Ocean City
416,416,William Kinna,2017-05-21,2017-05-26,2163.67,Conference,NC,Charlotte


In [38]:
#Using the .drop function from pandas, we define which column to drop and axis set 1 for column
travels_dataset = travels_dataset.drop('Unnamed: 0', axis=1)

In [40]:
travels_dataset #To view dataframe 

,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City
1,Abigail Ratnofsky,2016-01-24,2016-01-29,2962.70,Conference,CA,San Diego
2,Adam Kisthardt,2018-10-04,2018-10-09,1891.19,Conference,FL,Orlando
3,Adam P. Jones,2016-06-19,2016-06-21,324.36,Conference,MD,Ocean City
4,Alan Butsch,2019-11-19,2019-11-22,2141.51,Conference,CA,San Diego
...,...,...,...,...,...,...,...
413,Warren Jensen,2017-11-06,2017-11-09,502.45,Conference,MD,Ocean City
414,Warp Jensen,2019-03-25,2019-03-26,78.00,Conference,MD,Ocean City
415,Whitney Kujawa,2017-11-06,2017-11-09,567.00,Conference,MD,Ocean City
416,William Kinna,2017-05-21,2017-05-26,2163.67,Conference,NC,Charlotte


In [44]:
employees_expense = travels_dataset[['Employee', 'Actual Total Expenses']] # To retrieve Employee and Actual Total Expenses

In [47]:
employees_expense[employees_expense['Employee'].duplicated()]

,Employee,Actual Total Expenses
26,Barbara Natoli,991.12
34,Bill Ferretti,353.22
52,Catherine Brewer,4925.24
80,Darren Crandell,1995.22
99,Dimitry Ruvin,2195.32
189,John Bentley,1119.22
229,Kevin Parker,1899.98
255,Lissa Bales,258.64
259,Mackell Heflin,2964.20
274,Mark Sheelor,1411.34


In [55]:
#This will get the total expenses for each employee
employees_expense = employees_expense.groupby(by='Employee').sum()
employees_expense = employees_expense.reset_index() #This will reset the index to pandas' default

In [56]:

employees_df = employees_df.drop_duplicates(subset='name') #Drop duplicated names

In [58]:
result = employees_expense.merge(employees_df, how='inner', left_on='Employee', right_on='name')

In [59]:
result.head(10)

,Employee,Actual Total Expenses,name,stipend
0,Aaron Salter,644.10,Aaron Salter,7377
1,Abigail Ratnofsky,2962.70,Abigail Ratnofsky,6086
2,Adam Kisthardt,1891.19,Adam Kisthardt,7377
3,Adam P. Jones,324.36,Adam P. Jones,6832
4,Alan Butsch,2141.51,Alan Butsch,6913
5,Alberto Hook,1856.49,Alberto Hook,6913
6,Alberto Rein,330.08,Alberto Rein,6832
7,Alison Danielson,2346.12,Alison Danielson,6832
8,Alison Dollar,964.52,Alison Dollar,6832
9,Alison Wakefield,1050.08,Alison Wakefield,6832


In [64]:
result.drop(columns=['name'], inplace=True)  # This will drop the column named Destination from the dataframe)

In [65]:
result.head(10)

,Employee,Actual Total Expenses,stipend
0,Aaron Salter,644.10,7377
1,Abigail Ratnofsky,2962.70,6086
2,Adam Kisthardt,1891.19,7377
3,Adam P. Jones,324.36,6832
4,Alan Butsch,2141.51,6913
5,Alberto Hook,1856.49,6913
6,Alberto Rein,330.08,6832
7,Alison Danielson,2346.12,6832
8,Alison Dollar,964.52,6832
9,Alison Wakefield,1050.08,6832


In [68]:
#This will compare the actual expense of each employee with the allocated stipend and 
#get employees that spent more than their approved stipend.
final_result = result[result['Actual Total Expenses'] > result['stipend']] 

In [69]:
final_result #This is the answer to Question 1

,Employee,Actual Total Expenses,stipend
49,Catherine Brewer,8208.76,6913
372,Thomas Jordan,13247.80,7377
